# Pennylane: I. Introduction to Quantum Computing
## I.13. We've Got It Under Control

In [1]:
# preparation
import numpy as np
import pennylane as qml

### Prepare the States
This function prepares  a two-qubit state as a function of phi, theta, and omega. There are different ways of doing this. For now, we prepare as follows.

In [2]:
# define prepare_states function
def prepare_states(phi, theta, omega):
    # prepare 0
    qml.RX(phi, wires=0)
    qml.RY(theta, wires=0)
    qml.RZ(omega, wires=0)

    # make 00, 01
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[0, 1])

### Codercise I.13.1 - The imposter CZ

**Task**: Implement two methods for constructing the controlled-Z gate.

**Solution**: 
- Using `qml.CZ(wires=[control, target]`, we can create the CZ gate truly.
- For the imposter CZ, note that
$CZ=\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & -1
\end{pmatrix}$.
- Recall that
$$H=\frac{1}{\sqrt{2}}
\begin{pmatrix}
1 & 1\\
1 & -1
\end{pmatrix}, ~~~
CNOT=\begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0
\end{pmatrix}$$
- When we implemented the Z gate, we sandwiched the X gate between two Hadamard gates.
- Now, we do it again, but use CNOT instead of X to make CZ.
$$CZ=H~~CNOT~~H$$

In [3]:
dev = qml.device("default.qubit", wires=2)

# Prepare a two-qubit state; change up the angles if you like
phi, theta, omega = 1.2, 2.3, 3.4


@qml.qnode(device=dev)
def true_cz(phi, theta, omega):
    prepare_states(phi, theta, omega)

    ##################
    # YOUR CODE HERE #
    ##################

    # IMPLEMENT THE REGULAR CZ GATE HERE
    qml.CZ(wires=[0, 1])
    return qml.state()


@qml.qnode(dev)
def imposter_cz(phi, theta, omega):
    prepare_states(phi, theta, omega)

    ##################
    # YOUR CODE HERE #
    ##################

    # IMPLEMENT CZ USING ONLY H AND CNOT
    qml.Hadamard(0)
    qml.CNOT(wires=[1, 0]) # control: [1] # target: [0]
    qml.Hadamard(0)
    return qml.state()


print(f"True CZ output state {true_cz(phi, theta, omega)}")
print(f"Imposter CZ output state {imposter_cz(phi, theta, omega)}")

True CZ output state [ 0.33067914-0.2833615j   0.33067914-0.2833615j   0.09310021+0.54926307j
 -0.09310021-0.54926307j]
Imposter CZ output state [ 0.33067914-0.2833615j   0.33067914-0.2833615j   0.09310021+0.54926307j
 -0.09310021-0.54926307j]


### Codercise I.13.2 - The SWAP gate

**Task**: The SWAP gate can be implemented using CNOT gates.

<img height="20%" width="20%" src="https://assets.cloud.pennylane.ai/codebook/swap.svg"/>

**Solution**: Using `qml.SWAP(wires=[control, target])`, we can create the SWAP gate directly. Similarly, we can use 3 CNOT gates to perform the same operation. The middle CNOT has to be placed in opposite direction with respect to outer two CNOT gates.

In [4]:
dev = qml.device("default.qubit", wires=2)

# Prepare a two-qubit state; change up the angles if you like
phi, theta, omega = 1.2, 2.3, 3.4


@qml.qnode(dev)
def apply_swap(phi, theta, omega):
    prepare_states(phi, theta, omega)

    ##################
    # YOUR CODE HERE #
    ##################

    # IMPLEMENT THE REGULAR SWAP GATE HERE
    qml.SWAP(wires=[0, 1])
    return qml.state()


@qml.qnode(dev)
def apply_swap_with_cnots(phi, theta, omega):
    prepare_states(phi, theta, omega)

    ##################
    # YOUR CODE HERE #
    ##################

    # IMPLEMENT THE SWAP GATE USING A SEQUENCE OF CNOTS
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 0])
    qml.CNOT(wires=[0, 1])
    return qml.state()


print(f"Regular SWAP state = {apply_swap(phi, theta, omega)}")
print(f"CNOT SWAP state = {apply_swap_with_cnots(phi, theta, omega)}")

Regular SWAP state = [0.33067914-0.2833615j  0.09310021+0.54926307j 0.33067914-0.2833615j
 0.09310021+0.54926307j]
CNOT SWAP state = [0.33067914-0.2833615j  0.09310021+0.54926307j 0.33067914-0.2833615j
 0.09310021+0.54926307j]


### Codercise I.13.3 - The Toffoli gate

<img height="15%" width="15%" src="https://assets.cloud.pennylane.ai/codebook/toffoli.svg"/>

The gate shown above is called the **Toffoli gate**. It flips a target qubit if two control-inputs are in the state $|1\rangle$.

For a three-qubit system, its mathematical description is $TOF|abc\rangle=|ab(c\otimes ab)\rangle$.

**Task**: Implement the **Fredkin gate**, also known as the **controlled SWAP** operation.

<img height="15%" width="15%" src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/98/Qcircuit_Fredkin.svg/1200px-Qcircuit_Fredkin.svg.png"/>

**Solution**: Using `qml.Toffoli(wires=[control1, control2, target]`, we can create a 2-qubit controlled gate. To create the controlled SWAP gate, we need 3 Toffoli gates in a sequence. The implementation of the controlled SWAP gate is similar to the implementation of the SWAP gate. The middle Toffoli has to be placed in opposite direction while being sandwiched by the remaining 2 Toffolis.

In [5]:
dev = qml.device("default.qubit", wires=3)

# Prepare first qubit in |1>, and arbitrary states on the second two qubits
phi, theta, omega = 1.2, 2.3, 3.4


# A helper function just so you can visualize the initial state
# before the controlled SWAP occurs.
@qml.qnode(dev)
def no_swap(phi, theta, omega):
    prepare_states(phi, theta, omega)
    return qml.state()


@qml.qnode(dev)
def controlled_swap(phi, theta, omega):
    prepare_states(phi, theta, omega)

    ##################
    # YOUR CODE HERE #
    ##################

    # PERFORM A CONTROLLED SWAP USING A SEQUENCE OF TOFFOLIS
    qml.Toffoli(wires=[0, 1, 2])
    qml.Toffoli(wires=[0, 2, 1])
    qml.Toffoli(wires=[0, 1, 2])
    return qml.state()


print(no_swap(phi, theta, omega))
print(controlled_swap(phi, theta, omega))

[0.33067914-0.2833615j  0.        +0.j         0.33067914-0.2833615j
 0.        +0.j         0.09310021+0.54926307j 0.        +0.j
 0.09310021+0.54926307j 0.        +0.j        ]
[0.33067914-0.2833615j  0.        +0.j         0.33067914-0.2833615j
 0.        +0.j         0.09310021+0.54926307j 0.09310021+0.54926307j
 0.        +0.j         0.        +0.j        ]


### Codercise I.13.3 - Mixed Controlled Gates

In some cases, we want to control on a qubit being in the state $|0\rangle$. In PennyLane, `MultiControlledX()` operation can create mixed-polarity multi-controlled Toffoli gates. This gate allows us to define an arbitary bit-string for control.

**Task**: Write a 4-qubit PennyLane circuit that applies a Hadamard to the control qubits, then applies a `MultiControlledX()` on the fourth qubit, controlled on the first 3 qubits being in the state $001|\rangle$. This is depicted in the circuit below: "control on 0" is denoted by an open circle on the control qubits, rather than a filled circle. What do you expect will happen to the target qubit?

<img height="15%" width="15%" src="https://assets.cloud.pennylane.ai/codebook/mcx.svg"/>

**Solution**: This is a simple implmentation.

In [6]:
dev = qml.device("default.qubit", wires=4)


@qml.qnode(dev)
def four_qubit_mcx():
    ##################
    # YOUR CODE HERE #
    ##################

    # IMPLEMENT THE CIRCUIT ABOVE USING A 4-QUBIT MULTI-CONTROLLED X
    qml.Hadamard(0)
    qml.Hadamard(1)
    qml.Hadamard(2)    
    qml.MultiControlledX(wires=[0, 1, 2, 3], control_values=[0, 0, 1]) # control: [0], [1], [2] # target: [3] # control when "001"
    return qml.state()


print(four_qubit_mcx())

[0.35355339+0.j 0.        +0.j 0.        +0.j 0.35355339+0.j
 0.35355339+0.j 0.        +0.j 0.35355339+0.j 0.        +0.j
 0.35355339+0.j 0.        +0.j 0.35355339+0.j 0.        +0.j
 0.35355339+0.j 0.        +0.j 0.35355339+0.j 0.        +0.j]


### Codercise I.13.5 - The 3-controlled-NOT

As an example, suppose we have a four-qubit circuit.

Three Hadamard gates in this circuit put the first 3 qubits to be in a linear superposition.
$$(H\otimes H\otimes H)|000\rangle=\frac{1}{\sqrt{2}^3}(|000\rangle+|001\rangle+...+|111\rangle)$$

With an additional Identity gate, the last qubit is not changed. It is still in the state $|0\rangle$.
$$(H\otimes H\otimes H\otimes I)|0000\rangle=\frac{1}{\sqrt{2}^3}(|0000\rangle+|0010\rangle+...+|1110\rangle)$$

If the state of the first 3 qubits is $|001\rangle$, the multi-controlled gate (MCX) is triggered. When MCX is triggered, it flips the fourth qubit for that term.
- $|0010\rangle$ --> MCX --> $|0011\rangle$
- $|0011\rangle$ --> MCX --> $|0010\rangle$

The remaining gates are not affected.
$$MCX(H\otimes H\otimes H\otimes I)|0000\rangle=\frac{1}{\sqrt{2}^3}(|0000\rangle+|0011\rangle+...+|1110\rangle)$$

**Task**: Implement this 3-controlled-NOT gate using only Toffolis. An auxiliary qubit, initiated in $|0\rangle$, is necessary for this. And this auxiliary qubit must remain its initial state at the end of the circuit. 

<img height="60%" width="60%" src="https://assets.cloud.pennylane.ai/codebook/4cx.svg"/>

**Solution**: A total of 3 Toffoli gates are required. The first Toffoli gate flips the auxiliary qubit to $|1\rangle$. The middle Toffoli gate takes that auxiliary and another control qubits as inputs to set the target qubit. The last Toffoli gate resets (flips again) the auxiliary qubit to $|0\rangle$.

In [7]:
# Wires 0, 1, 2 are the control qubits
# Wire 3 is the auxiliary qubit
# Wire 4 is the target
dev = qml.device("default.qubit", wires=5)


@qml.qnode(dev)
def four_qubit_mcx_only_tofs():
    # We will initialize the control qubits in state |1> so you can see
    # how the output state gets changed.
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    qml.PauliX(wires=2)

    ##################
    # YOUR CODE HERE #
    ##################

    # IMPLEMENT A 3-CONTROLLED NOT WITH TOFFOLIS
    qml.Toffoli(wires=[0, 1, 3]) # flip the auxiliary
    qml.Toffoli(wires=[2, 3, 4]) # flip the target wire 4
    qml.Toffoli(wires=[0, 1, 3]) # flip the auxiliary
    return qml.state()


print(four_qubit_mcx_only_tofs())

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j
 0.+0.j 0.+0.j]


This notebook is done by `Myanmar Youths` for `Womanium Quantum + AI 2024` program.
- <a href="https://www.linkedin.com/in/la-wun-nannda-b047681b5/"><u>La Wun Nannda</u></a>
- <a href="https://www.linkedin.com/in/chit-zin-win-46a2a3263/"><u>Chit Zin Win</u></a>